In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import torch
from tqdm import tqdm
import gc
from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse, r2_score
from sklearn.model_selection import train_test_split




In [2]:
train_df = pd.read_csv('./Train.csv')
# train_df = train_df.drop(['agreement'], axis=1)

In [3]:
train_df.loc[~train_df['label'].isin([0, -1, 1]), 'label'] = -1
train_df.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    2
dtype: int64

In [4]:
train_df = train_df.dropna()
train_df.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

In [6]:
trains_df, evals_df = train_test_split(train_df, test_size=0.2, random_state=2)


In [7]:
train_data = trains_df[['safe_text', 'label']]
eval_data = evals_df[['safe_text', 'label']]

In [8]:
model = ClassificationModel('roberta', 'roberta-large', num_labels=1, args={
            'train_batch_size': 16,
            'eval_batch_size': 16,
            'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'fp16': True,
            'do_lower_case': False,
            'num_train_epochs': 3,
            'max_seq_length': 236,
            'regression': True,
            'manual_seed': 2,
            'learning_rate': 1e-5,
            'save_eval_checkpoints': False,
            'save_model_every_epoch': False,
        })

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [9]:
model.train_model(train_data)
preds_val = model.eval_model(eval_data)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(eval_data['label'], preds_val)**0.5}")

c:\Users\user\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/7999 [00:00<?, ?it/s]

In [ ]:
test_df = pd.read_csv('./Test.csv')
test_df['safe_text'] = test_df['safe_text'].fillna('xxxxxx')

In [ ]:
tmp_test = test_df[['safe_text']]
tmp_test['labels'] = 0


In [ ]:
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)

In [ ]:
test_preds_df = pd.DataFrame({'id': test_df['tweet_id'], 'label': test_preds})

test_preds_df.to_csv('Submission1.csv', index=False)
